In [21]:
!!pip install anthropic


['Collecting anthropic',
 '  Downloading anthropic-0.68.0-py3-none-any.whl (325 kB)',
 '     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 3.8 MB/s eta 0:00:00',
 'Requirement already satisfied: sniffio in /Users/lindazhou/opt/anaconda3/envs/myenv/lib/python3.8/site-packages (from anthropic) (1.3.0)',
 'Collecting distro<2,>=1.7.0',
 '  Downloading distro-1.9.0-py3-none-any.whl (20 kB)',
 'Collecting docstring-parser<1,>=0.15',
 '  Downloading docstring_parser-0.17.0-py3-none-any.whl (36 kB)',
 'Collecting jiter<1,>=0.4.0',
 '  Downloading jiter-0.9.1-cp38-cp38-macosx_10_12_x86_64.whl (313 kB)',
 '     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.5/313.5 kB 24.9 MB/s eta 0:00:00',
 'Collecting httpx<1,>=0.25.0',
 '  Downloading httpx-0.28.1-py3-none-any.whl (73 kB)',
 '     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 8.0 MB/s eta 0:00:00',
 'Requirement already satisfied: anyio<5,>=3.5.0 in /Users/lindazhou/opt/anaconda3/envs/myenv/lib/python3.8/site-packages (from ant

In [11]:
!pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 10.2 MB/s eta 0:00:00


In [7]:
tmp_pmid = "19686436"

In [15]:
from pathlib import Path

# define the file path
file_path = Path("papers 250") / tmp_pmid / f"{tmp_pmid}.checked.md"

# read the file contents into a string
with file_path.open("r", encoding="utf-8") as f:
    paper_content = f.read()

print(paper_content[:500])   # preview first 500 chars


# Prevalence of drug resistance and importance of viral load measurements in Honduran HIV-infected patients failing antiretroviral treatment

## Abstract

### Objective

The Honduran HIV/AIDS Program began to scale up access to HIV therapy in 2002. Up to May 2008, more than 6000 patients received combination antiretroviral therapy (cART). As HIV drug resistance is the major obstacle for effective treatment, the purpose of this study was to assess the prevalence of antiretroviral drug resistance 


In [14]:
import pandas as pd

# read the Excel file, first sheet
df = pd.read_excel("Fine-tuning instruction set, Aug 22.xlsx", sheet_name=0)

# filter rows where PMID = 19686436
filtered_df = df[df["PMID"] == int(tmp_pmid)]

print(filtered_df)


          PMID  QID                                           Question  \
0     19686436    1  Does the paper report HIV sequences from patie...   
250   19686436    2  Does the paper report in vitro drug susceptibi...   
500   19686436    3  What were the GenBank accession numbers for se...   
750   19686436    4       Which HIV species were studied in the paper?   
1000  19686436    5  Which HIV genes were reported to have been seq...   
1250  19686436    6  From which countries were the sequenced sample...   
1500  19686436    7  From what years were the sequenced samples obt...   
1750  19686436    8               What method was used for sequencing?   
2000  19686436    9           Were samples cloned prior to sequencing?   
2250  19686436   10      Did samples undergo single genome sequencing?   
2500  19686436   11               What type of samples were sequenced?   
2750  19686436   12  Were any sequences obtained from individuals w...   
3000  19686436   13  Were any sequence

In [17]:
with open("explain_multi_questions.txt", "r", encoding="utf-8") as f:
    template = f.read()
    
# --- Step 4: Create prompts per question ---
prompts = []
for _, row in filtered_df.iterrows():
    question = row["Question"]
    prompt = template.format(
        paper_content=paper_content,
        question=question
    )
    prompts.append({
        "QID": row["QID"],
        "PMID": row["PMID"],
        "prompt": prompt
    })

print(prompts[0])

{'QID': 1, 'PMID': 19686436, 'prompt': 'Read the paper in "Paper content" section, and answer a list of questions in "Questions" section below,\n\n## For each question:\n\nStep 1: get the question, store as "question".\nStep 2: extract two or three sentences from the "paper content" that can be used to answer the question, separate them using \'.\', store as \'evidence\'.\nStep 3: provide the rationale about how you found the answer from the content in details, store as \'rationale\'.\nStep 4: answer the question, store as \'answer\'.\nStep 5: format your answer in the format:\n\n"""\nQuestion: <question>\n\nEvidence: <evidence>\n\nRationale: <rationale>\n\nAnswer: <answer>\n"""\n\nMake sure you answer all the questions.\n\n\n## Paper content:\n\n```\n# Prevalence of drug resistance and importance of viral load measurements in Honduran HIV-infected patients failing antiretroviral treatment\n\n## Abstract\n\n### Objective\n\nThe Honduran HIV/AIDS Program began to scale up access to HIV 

In [18]:
questions_block = ""
for _, row in filtered_df.iterrows():
    qid = row["QID"]
    qtext = row["Question"]
    questions_block += f"{qid}. {qtext}\n"

# --- Step 5: Fill the template ---
final_prompt = template.format(
    paper_content=paper_content,
    question=questions_block.strip()
)

# --- Step 6: Save final prompt ---
output_file = Path("19686436_all_questions_prompt.txt")
with output_file.open("w", encoding="utf-8") as f:
    f.write(final_prompt)
print(final_prompt)

Read the paper in "Paper content" section, and answer a list of questions in "Questions" section below,

## For each question:

Step 1: get the question, store as "question".
Step 2: extract two or three sentences from the "paper content" that can be used to answer the question, separate them using '.', store as 'evidence'.
Step 3: provide the rationale about how you found the answer from the content in details, store as 'rationale'.
Step 4: answer the question, store as 'answer'.
Step 5: format your answer in the format:

"""
Question: <question>

Evidence: <evidence>

Rationale: <rationale>

Answer: <answer>
"""

Make sure you answer all the questions.


## Paper content:

```
# Prevalence of drug resistance and importance of viral load measurements in Honduran HIV-infected patients failing antiretroviral treatment

## Abstract

### Objective

The Honduran HIV/AIDS Program began to scale up access to HIV therapy in 2002. Up to May 2008, more than 6000 patients received combination an

In [22]:
from anthropic import Anthropic

client = Anthropic(api_key=claude_api_key)

response = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=2000,
    messages=[
        {"role": "user", "content": final_prompt}
    ]
)

answer = response.content[0].text

# --- Step 1: Token usage ---
input_tokens = response.usage.input_tokens
output_tokens = response.usage.output_tokens
total_tokens = input_tokens + output_tokens

print(f"Input tokens: {input_tokens}")
print(f"Output tokens: {output_tokens}")
print(f"Total tokens: {total_tokens}")

# --- Step 2: Pricing (Claude 3.5 Sonnet as of 2024-09) ---
# Input: $3.00 per 1M tokens = $0.000003 per token
# Output: $15.00 per 1M tokens = $0.000015 per token
input_cost = input_tokens * 0.000003
output_cost = output_tokens * 0.000015
total_cost = input_cost + output_cost

print(f"Estimated cost: ${total_cost:.4f}")


/var/folders/jk/qtmyn4k16f15cydbjzyrmpv00000gn/T/ipykernel_98626/1928519445.py:5: DeprecationWarning: The model 'claude-3-5-sonnet-20240620' is deprecated and will reach end-of-life on October 22, 2025.
Please migrate to a newer model. Visit https://docs.anthropic.com/en/docs/resources/model-deprecations for more information.
  response = client.messages.create(


Input tokens: 6758
Output tokens: 2000
Total tokens: 8758
Estimated cost: $0.0503


In [23]:
import re
# --- Step 2: Split Claude’s output into blocks ---
# Matches the format:
# Question: ...
# Evidence: ...
# Rationale: ...
# Answer: ...
pattern = r"Question:(.*?)\n\nEvidence:(.*?)\n\nRationale:(.*?)\n\nAnswer:(.*?)(?=\n\nQuestion:|$)"
matches = re.findall(pattern, answer, re.DOTALL)

# --- Step 3: Build dataframe ---
records = []
filtered_df = df[df["PMID"] == 19686436]  # your filtered 18 rows

for i, (q, ev, rat, ans) in enumerate(matches):
    # strip extra whitespace
    q, ev, rat, ans = q.strip(), ev.strip(), rat.strip(), ans.strip()

    # match to QID + original Question
    if i < len(filtered_df):
        row = filtered_df.iloc[i]
        records.append({
            "PMID": row["PMID"],
            "QID": row["QID"],
            "Question": q,
            "Evidence": ev,
            "Rationale": rat,
            "Answer": ans
        })

output_df = pd.DataFrame(records)

# --- Step 4: Save to Excel ---
output_df.to_excel("claude_answers_19686436.xlsx", index=False)

print("✅ Saved answers to claude_answers_19686436.xlsx")

✅ Saved answers to claude_answers_19686436.xlsx
